<h6>PE02_A02<h6>
    
# SBBFU Project 

### 1. About the project 
 
    
##### 
    
> We aim to develop Python-based data analysis software
> The goal is to develop software that can analyze wafer scale data in detail.
>
> At first,
>
> The main task is to receive data from the customer and then receive the request.
> 
> Main request 
> + Lot
> + Wafer
> + Die row & colum
> 
> We can analyze the selected data by specifying specific elements    
> In order to develop such analysis software, 
> a black box(software) that implments this function is created to solve the customer's request.
>
> In the software,
> We process the raw data and print it out. 
> We show the processed data as a 2*3 figure and save the file. 
> 
> In addition, it creates an Excel csv file that can analyze varous 
> xml files as a result file and makes a dataframe that can be compared simply.
>
> Finally, the final report The final goal is to make documentation about the tasks 
> we did using a jupyter notebook and report it to the customer.
>
> Please put the data in the 'Data' folder
> specify the file path
> Run the Run.py 

### 2. Run file description 
 
##### 
 >Create a folder corresponding to the output that can be delivered to customers by copying the data in the **'Data'** folder to **'Result'**
 >Data is processed by selecting only files with the filename "LMZ"
 >Load the module that processes and fits raw data and the module that makes the data parsed from xml file into a csv file, respectively.

In [ ]:
from src import process
import os
import glob

file_path = '.\dat\P184640\D07\**\*LMZ?.xml'
# xml = []
# for filename in glob.glob(file_path, recursive=True):
#     xml.append(filename)
    
# for i in xml:
procwssfitting('.\dat\P184640\D07\20190715_190855\P184640_D07_(0,0)_GORILLA5_DCM_LMZC.xml',1)